<a href="https://colab.research.google.com/gist/GunjanGrunge/3eac49e0a28ed64fd3ce6c8498cc0d36/tabular-playground-series-oct-2021-with-nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!kaggle competitions download tabular-playground-series-oct-2021

In [ ]:
def reduce_mem_usage(df, verbose=True):
    """
    function to reduce memory

    """
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                else:
                    df[col] = df[col].astype(np.float32)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#pip install datatable

In [ ]:
import random
random.seed(123)

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
import datatable as dt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
"""
train = dt.fread(r'../input/tabular-playground-series-oct-2021/train.csv').to_pandas()
test = dt.fread(r'../input/tabular-playground-series-oct-2021/test.csv').to_pandas()
sub = dt.fread(r'../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()
""" 

In [ ]:
train = dt.fread(r'/content/train.csv.zip').to_pandas()
test = dt.fread(r'/content/test.csv.zip').to_pandas()
sub = dt.fread(r'/content/sample_submission.csv.zip').to_pandas()

In [ ]:
train_1 = reduce_mem_usage(train)
test_1= reduce_mem_usage(test)

### Using Train and test with reduced memory 

In [ ]:
train_1.head()

In [ ]:
train_1.isnull().sum()

In [ ]:
pd.set_option('display.max_columns', 125)
train_1.describe()

In [ ]:
#converting the Target column to numrcal values i.r 0 or 1. 
train_1.target = train_1.target.astype('int16')
train_1.info()
print()
test_1.info()

In [ ]:
from sklearn.model_selection import train_test_split

X = train_1.drop("target", axis=1)
y = train_1['target']

In [ ]:
print(train_1.target.dtype)
print(train_1.target[:10])



In [ ]:
X = train_1 
Y = X.target
X.drop('target', axis = 1, inplace = True)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2,
                                                                         random_state = 42)
X_test = test_1

In [ ]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean', verbose = False)
my_imputer.fit(X_train)

In [ ]:
X_train_imputed = my_imputer.transform(X_train)


In [ ]:
X_valid_imputed = my_imputer.transform(X_val)


In [ ]:
X_test_imputed = my_imputer.transform(X_test)

In [ ]:
standard_scaler = StandardScaler()
standard_scaler.fit(X_train_imputed)

In [ ]:
X_train_scaled = standard_scaler.transform(X_train_imputed)


In [ ]:
X_valid_scaled = standard_scaler.transform(X_valid_imputed)
X_test_scaled = standard_scaler.transform(X_test_imputed)

In [ ]:
#Converting the numpy array to dataframe as standard scaler transforms the dataframe to Np Array

In [ ]:
X_train_final = pd.DataFrame(X_train_scaled, index = X_train.index,
                            columns = X_train.columns)

In [ ]:
X_valid_final = pd.DataFrame(X_valid_scaled, index = X_val.index, 
                            columns = X_val.columns)
X_test_final = pd.DataFrame(X_test_scaled, index = X_test.index, 
                           columns = X_test.columns)

In [ ]:
X_train_final.describe().T

In [ ]:
nn = keras.Sequential([
    layers.BatchNormalization(input_shape = [X_train_final.shape[1]]),
    layers.Dense(units = 200, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate = 0.2),
    layers.Dense(units = 100, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate = 0.2),
    layers.Dense(units = 50, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate = 0.2),
    layers.Dense(units = 10, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate = 0.2),
    layers.Dense(units = 1, activation = 'sigmoid')
])

In [ ]:


tf.keras.utils.plot_model(nn, show_shapes=True)


In [ ]:
nn.summary()

In [ ]:
auc = keras.metrics.AUC(name = 'auc')
nn.compile(optimizer = keras.optimizers.Adam(learning_rate = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 0.01, decay_steps = 9000, decay_rate = 0.9)),
             loss = 'binary_crossentropy',
             metrics = [auc])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience = 10,
                                min_delta = 0.001,
                                restore_best_weights = False)

In [ ]:
history = nn.fit(X_train_final, Y_train,
                   validation_data = (X_valid_final,Y_val),
                   batch_size = 500,
                   epochs = 50, callbacks = [early_stopping])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot(title = 'Loss')
history_df.loc[:,['auc','val_auc']].plot(title = 'Dev')

In [ ]:
X_test_final.shape

In [ ]:
preds = nn.predict(X_test_final)

In [ ]:
preds = preds.reshape(len(X_test_final),)

In [ ]:
output = pd.DataFrame({'id' : X_test_final.index,
                      'target' : preds})
output.to_csv('submission.csv', index = False)